In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
!pip install lightgbm

!pip install lightgbm --upgrade

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/풍력 발전_dacon/train.csv')
test = pd.read_csv('/content/drive/MyDrive/풍력 발전_dacon/test.csv')

In [ ]:
train.head()

,id,temperature,pressure,humidity,wind_speed,wind_direction,precipitation,snowing,cloudiness,target
0,TRAIN_00000,27.8816,1013.6,46.8,2.4,232.4,0.00,False,0.0,1.180
1,TRAIN_00001,5.7540,1026.6,76.6,3.6,172.0,0.00,False,8.8,8.581
2,TRAIN_00002,20.8220,1016.2,64.8,2.2,206.0,0.06,False,23.0,3.978
3,TRAIN_00003,20.0758,1017.4,72.6,0.8,215.4,0.00,False,0.0,8.301
4,TRAIN_00004,7.5260,1023.2,82.8,1.2,158.0,0.00,False,15.0,1.692


In [ ]:
index_1 = train[train['pressure'] <=850].index
for i in index_1:
  train['pressure'][i] = 1016.152817
index_1 = test[test['pressure'] <=850].index
for i in index_1:
  test['pressure'][i] = 1016.152817


index_1 = train[train['pressure'] >=1100].index
for i in index_1:
  train['pressure'][i] = 1016.152817

index_2 = test[test['pressure'] >=1200].index
for i in index_2:
  test['pressure'][i] = 1016.152817

<ipython-input-166-27795b154129>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pressure'][i] = 1016.152817
<ipython-input-166-27795b154129>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pressure'][i] = 1016.152817
<ipython-input-166-27795b154129>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pressure'][i] = 1016.152817
<ipython-input-166-27795b154129>:15: SettingWithCopyWarning: 
A value is trying to be se

In [ ]:
# X는 독립변수이므로 종속변수를 제거합니다. 또한 target 이외의 분석에 활용하지 않는 데이터(id)를 제거합니다.
train_x = train.drop(columns=['id', 'target', 'snowing'])
# y는 종속변수로 값을 설정합니다.
train_y = train['target']

In [ ]:
# train에서와 마찬가지로 분석에 활용하지 않는 데이터(id)를 제거합니다.
test_x = test.drop(columns=['id', 'snowing'])

In [ ]:
test_x

,temperature,pressure,humidity,wind_speed,wind_direction,precipitation,cloudiness
0,20.702000,1021.0,76.4,2.0,104.8,0.00,23.0
1,18.673669,1014.4,59.6,2.4,179.2,0.12,32.4
2,29.908000,1020.6,49.6,2.6,78.0,0.60,23.0
3,24.504000,1014.6,61.0,1.8,118.0,0.00,15.0
4,12.590000,1011.4,70.8,1.8,110.0,0.12,34.0
...,...,...,...,...,...,...,...
19270,25.180000,1020.2,64.8,1.6,122.0,0.00,34.0
19271,16.940000,1022.6,72.4,1.8,245.6,0.00,19.0
19272,9.032000,1036.4,94.4,2.2,219.2,0.00,18.0
19273,15.774000,1021.8,79.0,0.8,182.0,0.00,27.0


In [ ]:
# Min-Max 스케일링 객체 생성
scaler = MinMaxScaler()

# 스케일링 적용
scaled_data = scaler.fit_transform(train_x)

# 스케일링 결과 출력
print(scaled_data)

[[0.75376037 0.75706215 0.3126615  ... 0.68757396 0.         0.        ]
 [0.15998807 0.87947269 0.69767442 ... 0.50887574 0.         0.09401709]
 [0.56432297 0.78154426 0.54521964 ... 0.60946746 0.01886792 0.2457265 ]
 ...
 [0.37449917 0.62711864 0.54780362 ... 0.76923077 0.11320755 0.56623932]
 [0.52493064 0.81167608 0.29457364 ... 0.35502959 0.01886792 0.21367521]
 [0.37406613 0.66854991 0.7622739  ... 0.65739645 0.01886792 0.47222222]]


In [ ]:
test_x

,temperature,pressure,humidity,wind_speed,wind_direction,precipitation,cloudiness
0,20.702000,1021.0,76.4,2.0,104.8,0.00,23.0
1,18.673669,1014.4,59.6,2.4,179.2,0.12,32.4
2,29.908000,1020.6,49.6,2.6,78.0,0.60,23.0
3,24.504000,1014.6,61.0,1.8,118.0,0.00,15.0
4,12.590000,1011.4,70.8,1.8,110.0,0.12,34.0
...,...,...,...,...,...,...,...
19270,25.180000,1020.2,64.8,1.6,122.0,0.00,34.0
19271,16.940000,1022.6,72.4,1.8,245.6,0.00,19.0
19272,9.032000,1036.4,94.4,2.2,219.2,0.00,18.0
19273,15.774000,1021.8,79.0,0.8,182.0,0.00,27.0


In [ ]:

x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

# LGBMRegressor 객체 생성 (회귀 모델로 변경)
lgbm_wrapper = LGBMRegressor(n_estimators=4000)

# 학습 (조기 중단 지정)
lgbm_wrapper.fit(x_train, y_train)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002156 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1299
[LightGBM] [Info] Number of data points in the train set: 15420, number of used features: 7
[LightGBM] [Info] Start training from score 5.466626


LGBMRegressor(n_estimators=4000)

In [ ]:
test_x


,temperature,pressure,humidity,wind_speed,wind_direction,precipitation,cloudiness
0,20.702000,1021.0,76.4,2.0,104.8,0.00,23.0
1,18.673669,1014.4,59.6,2.4,179.2,0.12,32.4
2,29.908000,1020.6,49.6,2.6,78.0,0.60,23.0
3,24.504000,1014.6,61.0,1.8,118.0,0.00,15.0
4,12.590000,1011.4,70.8,1.8,110.0,0.12,34.0
...,...,...,...,...,...,...,...
19270,25.180000,1020.2,64.8,1.6,122.0,0.00,34.0
19271,16.940000,1022.6,72.4,1.8,245.6,0.00,19.0
19272,9.032000,1036.4,94.4,2.2,219.2,0.00,18.0
19273,15.774000,1021.8,79.0,0.8,182.0,0.00,27.0


In [ ]:
preds = lgbm_wrapper.predict(test_x)


In [ ]:
# 제출 파일을 불러옵니다.
submit = pd.read_csv('/content/drive/MyDrive/풍력 발전_dacon/sample_submission.csv')

In [ ]:

submit['target'] = preds
submit

,id,target
0,TEST_00000,6.287644
1,TEST_00001,4.463145
2,TEST_00002,7.899077
3,TEST_00003,4.743976
4,TEST_00004,7.468135
...,...,...
19270,TEST_19270,4.303644
19271,TEST_19271,4.603346
19272,TEST_19272,2.123236
19273,TEST_19273,5.622384


In [ ]:
submit.to_csv('/content/drive/MyDrive/풍력 발전_dacon/sample_submission.csv', index=False)